# AutoTSForecast Tutorial

Welcome to **AutoTSForecast** — an automated time series forecasting library that follows the sklearn API pattern.

## Quick Install

```bash
pip install autotsforecast
```

## What You'll Learn

In this tutorial, you'll learn how to:

1. **AutoForecaster vs Individual Models** — See how per-series model selection outperforms any single model
2. **Hierarchical Reconciliation** — Ensure coherent forecasts with region-level accuracy improvements
3. **Interpretability** — Understand which features drive your forecasts

## Dataset: Retail Sales with Covariates

We'll use a **synthetic retail sales dataset** with:
- **3 time series**: `region_a`, `region_b`, `total` (hierarchical structure)
- **2 covariates**: `temperature`, `promotion`
- **Different patterns**: Region A responds strongly to promotions, Region B to temperature

## Key Features Demonstrated

- **AutoForecaster superiority**: Outperforms Prophet, ARIMA, XGBoost, and other models
- **Per-Series Model Selection**: Each series independently selects its optimal model via CV
- **No Data Leakage**: Holdout test set is never touched during model selection
- **Region-Level Improvements**: Hierarchical reconciliation improves regional accuracy
- **Interpretability**: Sensitivity analysis and SHAP values

## Documentation Links

- [Quick Start](../QUICKSTART.md) — fastest overview
- [API Reference](../API_REFERENCE.md) — parameters and objects
- [README](../README.md) — package overview

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# AutoTSForecast imports
from autotsforecast import AutoForecaster
from autotsforecast.backtesting.validator import BacktestValidator
from autotsforecast.hierarchical.reconciliation import HierarchicalReconciler
from autotsforecast.interpretability.drivers import DriverAnalyzer

# Model imports
from autotsforecast.models.base import (
    LinearForecaster, 
    MovingAverageForecaster,
    VARForecaster
)
from autotsforecast.models.external import (
    ARIMAForecaster, 
    ETSForecaster, 
    ProphetForecaster,
    XGBoostForecaster,
    RandomForestForecaster,
    LSTMForecaster
)

# Optional: SHAP for interpretability
try:
    import shap
    SHAP_AVAILABLE = True
except ImportError:
    SHAP_AVAILABLE = False

# Set random seed for reproducibility
np.random.seed(42)

# Compatibility fixes for newer numpy versions
if not hasattr(np, "trapz") and hasattr(np, "trapezoid"):
    np.trapz = np.trapezoid
if not hasattr(np, "in1d") and hasattr(np, "isin"):
    np.in1d = np.isin

# Helper functions
def rmse(y_true, y_pred):
    """Calculate Root Mean Squared Error"""
    return float(np.sqrt(np.mean((np.asarray(y_true) - np.asarray(y_pred))**2)))

def mape(y_true, y_pred):
    """Calculate Mean Absolute Percentage Error"""
    return float(np.mean(np.abs((np.asarray(y_true) - np.asarray(y_pred)) / 
                                  (np.abs(np.asarray(y_true)) + 1e-9))) * 100)

print("✅ Setup complete!")

In [ ]:
# Create a sample retail sales dataset with covariates
np.random.seed(42)
n_days = 240
horizon = 14

# Generate dates
dates = pd.date_range('2023-01-01', periods=n_days, freq='D')
time_step = np.arange(n_days)

# Create covariates: temperature and promotions
temperature = 20 + 8 * np.sin(2 * np.pi * time_step / 7) + np.random.normal(0, 0.8, n_days)
promo = (np.random.rand(n_days) < 0.12).astype(int)
promo[-horizon:] = (np.random.rand(horizon) < 0.45).astype(int)
if promo[-horizon:].sum() == 0:
    promo[-1] = 1

X = pd.DataFrame({
    'temperature': temperature,
    'promotion': promo
}, index=dates)

# Generate sales data for two regions
shared_noise = np.random.normal(0, 4.0, n_days)

# Region A: Strong response to promotions
region_a_sales = (
    40 + 0.10 * time_step + 50.0 * X['promotion'].values + 
    1.6 * X['temperature'].values + shared_noise + np.random.normal(0, 0.8, n_days)
)

# Region B: Strong temperature effect
region_b_sales = (
    25 + 7.0 * np.sin(2 * np.pi * time_step / 30) + 2.0 * X['temperature'].values + 
    4.0 * X['promotion'].values - shared_noise + np.random.normal(0, 0.8, n_days)
)

# Total sales (hierarchical structure)
total_sales = region_a_sales + region_b_sales

y = pd.DataFrame({
    'region_a': region_a_sales,
    'region_b': region_b_sales,
    'total': total_sales
}, index=dates)

# Train/test split
y_train, y_test = y.iloc[:-horizon], y.iloc[-horizon:]
X_train, X_test = X.iloc[:-horizon], X.iloc[-horizon:]

print(f"📊 Dataset Overview:")
print(f"   Training: {y_train.index[0].date()} to {y_train.index[-1].date()} ({len(y_train)} days)")
print(f"   Test: {y_test.index[0].date()} to {y_test.index[-1].date()} ({len(y_test)} days)")
print(f"   Series: region_a, region_b, total")
print(f"   Covariates: temperature, promotion")

# Visualize the data
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

axes[0, 0].plot(y.index, y['region_a'], color='steelblue', linewidth=1.5)
axes[0, 0].axvline(y_train.index[-1], color='red', linestyle='--', alpha=0.7, label='Train/Test Split')
axes[0, 0].set_title('Region A Sales', fontweight='bold')
axes[0, 0].set_ylabel('Sales')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

axes[0, 1].plot(y.index, y['region_b'], color='darkorange', linewidth=1.5)
axes[0, 1].axvline(y_train.index[-1], color='red', linestyle='--', alpha=0.7, label='Train/Test Split')
axes[0, 1].set_title('Region B Sales', fontweight='bold')
axes[0, 1].set_ylabel('Sales')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

axes[1, 0].plot(y.index, y['total'], color='green', linewidth=1.5)
axes[1, 0].axvline(y_train.index[-1], color='red', linestyle='--', alpha=0.7, label='Train/Test Split')
axes[1, 0].set_title('Total Sales', fontweight='bold')
axes[1, 0].set_xlabel('Date')
axes[1, 0].set_ylabel('Sales')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

ax2 = axes[1, 1]
ax2.plot(X.index, X['temperature'], color='purple', alpha=0.7, linewidth=1.5)
ax2.axvline(X_train.index[-1], color='red', linestyle='--', alpha=0.7)
ax2.set_ylabel('Temperature (°C)', color='purple')
ax2.tick_params(axis='y', labelcolor='purple')
ax2.set_title('Covariates', fontweight='bold')
ax2.set_xlabel('Date')
ax2.grid(alpha=0.3)

ax2_right = ax2.twinx()
promo_dates = X[X['promotion'] == 1].index
ax2_right.scatter(promo_dates, X.loc[promo_dates, 'promotion'], color='red', alpha=0.6, s=30, marker='D')
ax2_right.set_ylabel('Promotion', color='red')
ax2_right.tick_params(axis='y', labelcolor='red')
ax2_right.set_ylim(-0.1, 1.1)

plt.tight_layout()
plt.show()

## Part 1: AutoForecaster vs Individual Models

This section demonstrates **AutoTSForecast's key advantage**: automatic per-series model selection that outperforms using any single model for all series.

### The Challenge

When forecasting multiple time series, you have two choices:
1. **Single Model**: Use the same model (e.g., Prophet) for all series
2. **Per-Series Selection**: Let each series pick its optimal model via CV

### AutoTSForecast Approach

```
┌─────────────────────────────────────────────────────────────────────┐
│                        TRAINING DATA ONLY                           │
│  ┌─────────────────────────────────────────────────────────────┐    │
│  │  For EACH series: Run CV with ALL candidate models          │    │
│  │  CV Fold 1:  [Train──────────────] [Val]                    │    │
│  │  CV Fold 2:  [Train────────────────────] [Val]              │    │
│  │  CV Fold 3:  [Train──────────────────────────] [Val]        │    │
│  └─────────────────────────────────────────────────────────────┘    │
│                           ↓                                         │
│         Select BEST model for EACH series (may differ!)             │
│                           ↓                                         │
│         Retrain best model on FULL training data                    │
└─────────────────────────────────────────────────────────────────────┘
                            ↓
┌─────────────────────────────────────────────────────────────────────┐
│                     HOLDOUT TEST SET                                │
│         (Never seen during training or model selection!)            │
│                           ↓                                         │
│         Evaluate: AutoForecaster vs each individual model           │
└─────────────────────────────────────────────────────────────────────┘
```

### Available Models

| Category | Models | Best For |
|----------|---------|----------|
| **Statistical** | Prophet, ARIMA, ETS | Trends & seasonality |
| **Baselines** | Linear, Moving Average | Simple benchmarks |
| **Multivariate** | VAR | Cross-series dependencies |
| **Machine Learning** | XGBoost, Random Forest | Complex patterns |
| **Deep Learning** | LSTM | Non-linear dynamics |

Below we compare **AutoForecaster** (per-series selection) against each individual model.

In [ ]:
# Part 1: AutoForecaster vs Individual Models
# This demonstrates AutoTSForecast's key advantage: per-series model selection
print("🔄 AutoForecaster vs Individual Models: Performance Comparison")
print("=" * 75)
print("""
💡 KEY TEST: Does AutoForecaster (per-series selection) outperform using
   ANY single model for all time series?
   
   We'll compare AutoForecaster against: Prophet, ARIMA, ETS, XGBoost, 
   RandomForest, LSTM, and MovingAverage.
   
   Models will use COVARIATES (temperature, promotion) where supported.
""")

# All available models in the package
all_models = {
    'Prophet': lambda: ProphetForecaster(horizon=horizon),
    'ARIMA': lambda: ARIMAForecaster(horizon=horizon),
    'ETS': lambda: ETSForecaster(horizon=horizon),
    'XGBoost': lambda: XGBoostForecaster(horizon=horizon, n_lags=14, n_estimators=100),
    'RandomForest': lambda: RandomForestForecaster(horizon=horizon, n_lags=14, n_estimators=100),
    'LSTM': lambda: LSTMForecaster(horizon=horizon, hidden_size=64, num_layers=2, epochs=50),
    'MovingAvg': lambda: MovingAverageForecaster(horizon=horizon, window=7),
}

# Models that support covariates
models_with_covariates = {'Prophet', 'ARIMA', 'XGBoost', 'RandomForest', 'LSTM'}

series_to_test = ['region_a', 'region_b', 'total']

print(f"📊 Testing {len(series_to_test)} time series × {len(all_models)} models")
print(f"   Series: {', '.join(series_to_test)}")
print(f"   Models: {', '.join(all_models.keys())}")
print(f"   With covariates: {', '.join(models_with_covariates)}")

# Step 1: Train AutoForecaster with per-series model selection
print("\n" + "=" * 75)
print("📊 STEP 1: Training AutoForecaster (Per-Series Model Selection)")
print("=" * 75)

# Create candidate models for AutoForecaster
candidate_models = [
    ProphetForecaster(horizon=horizon),
    ARIMAForecaster(horizon=horizon),
    ETSForecaster(horizon=horizon),
    XGBoostForecaster(horizon=horizon, n_lags=14, n_estimators=100),
    RandomForestForecaster(horizon=horizon, n_lags=14, n_estimators=100),
    LSTMForecaster(horizon=horizon, hidden_size=64, num_layers=2, epochs=50),
    MovingAverageForecaster(horizon=horizon, window=7),
]

# Train AutoForecaster with per_series_models=True
print(f"\n   🔹 Training AutoForecaster on all {len(series_to_test)} series...")
auto_forecaster = AutoForecaster(
    candidate_models=candidate_models,
    per_series_models=True,  # Enable per-series model selection
    n_splits=3,
    test_size=horizon,
    metric='rmse',
    verbose=False
)
auto_forecaster.fit(y_train, X=X_train)  # Fit on ALL series at once

# Display selected models
print(f"\n   ✅ Model selection complete:")
for series_name in series_to_test:
    best_model_name = type(auto_forecaster.best_models_[series_name]).__name__.replace('Forecaster', '')
    print(f"      • {series_name}: {best_model_name}")

print("\n✅ AutoForecaster training complete!")

# Step 2: Train individual models and compare on holdout test set
print("\n" + "=" * 75)
print("📊 STEP 2: Holdout Test Evaluation (AutoForecaster vs Individual Models)")
print("=" * 75)
print("   Training each individual model on full training data and testing on holdout...")

# Train and evaluate each individual model on holdout
holdout_matrix = {}
print(f"\n   {'Model':<15} {'region_a':<12} {'region_b':<12} {'total':<12} {'Avg RMSE':<12}")
print(f"   {'-'*65}")

for model_name, model_factory in all_models.items():
    holdout_matrix[model_name] = {}
    row_rmses = []
    
    for series_name in series_to_test:
        try:
            model = model_factory()
            # Use covariates if model supports them
            if model_name in models_with_covariates:
                model.fit(y_train[[series_name]], X=X_train)
                predictions = model.predict(X=X_test)
            else:
                model.fit(y_train[[series_name]])
                predictions = model.predict()
            test_rmse = rmse(y_test[series_name], predictions[series_name])
            holdout_matrix[model_name][series_name] = test_rmse
            row_rmses.append(test_rmse)
        except Exception as e:
            holdout_matrix[model_name][series_name] = np.nan
            row_rmses.append(np.nan)
    
    avg_rmse = np.nanmean(row_rmses) if row_rmses else np.nan
    holdout_matrix[model_name]['Average'] = avg_rmse
    
    print(f"   {model_name:<15} {holdout_matrix[model_name].get('region_a', np.nan):<12.2f} "
          f"{holdout_matrix[model_name].get('region_b', np.nan):<12.2f} "
          f"{holdout_matrix[model_name].get('total', np.nan):<12.2f} "
          f"{avg_rmse:<12.2f}")

# Get AutoForecaster results (uses per-series selected models)
print(f"   {'-'*65}")
auto_forecasts = auto_forecaster.forecast(X=X_test)
auto_rmses = []
auto_str = f"   ⭐ AutoForecaster "
for series_name in series_to_test:
    test_rmse = rmse(y_test[series_name], auto_forecasts[series_name])
    auto_rmses.append(test_rmse)
    auto_str += f" {test_rmse:<11.2f}"
auto_avg = np.mean(auto_rmses)
auto_str += f" {auto_avg:<12.2f}"
print(auto_str)

# Calculate improvements vs each model
print(f"\n" + "=" * 75)
print("📊 AUTOFORECASTER vs EACH INDIVIDUAL MODEL")
print("=" * 75)
print(f"\n   {'Model':<15} {'Model Avg':<12} {'AutoForecaster':<15} {'Improvement':<15} {'Result':<10}")
print(f"   {'-'*70}")

wins = 0
ties = 0
for model_name in all_models.keys():
    model_avg = holdout_matrix[model_name]['Average']
    improvement = model_avg - auto_avg
    pct_improvement = 100 * improvement / model_avg if model_avg > 0 else 0
    
    if improvement > 0.01:
        result = "✅ WINS"
        wins += 1
    elif improvement < -0.01:
        result = "❌ LOSES"
    else:
        result = "↔ TIE"
        ties += 1
    
    print(f"   {model_name:<15} {model_avg:<12.2f} {auto_avg:<15.2f} {improvement:>+6.2f} ({pct_improvement:>+5.1f}%) {result}")

# Summary
print(f"\n" + "=" * 75)
print("🏆 FINAL VERDICT: AutoForecaster Performance")
print("=" * 75)

# Show which models were selected by AutoForecaster
selected_models = {}
for series_name in series_to_test:
    selected_models[series_name] = type(auto_forecaster.best_models_[series_name]).__name__.replace('Forecaster', '')

print(f"""
   AutoForecaster (Per-Series Selection):
   • region_a → {selected_models['region_a']}
   • region_b → {selected_models['region_b']}
   • total    → {selected_models['total']}
   
   📊 Results vs Individual Models:
   • Wins: {wins}/{len(all_models)} models
   • Ties: {ties}/{len(all_models)} models
   • Average RMSE: {auto_avg:.2f}
      
   🎯 This is the core value proposition of AutoTSForecast:
      No need to manually test which model works best — let the algorithm
      find the optimal model for each series automatically!
""")

KeyboardInterrupt: 

## Part 2: Hierarchical Forecasting with Reconciliation

When forecasting at multiple aggregation levels (e.g., regions + total), independent forecasts often violate the constraint:

```
forecast(total) ≠ forecast(region_a) + forecast(region_b)
```

**Hierarchical reconciliation** adjusts forecasts to ensure coherence while **improving accuracy at the region level**.

### Why Reconciliation Helps Regions

OLS reconciliation doesn't just enforce coherence — it **optimally redistributes forecast errors** across all levels. This often means:
- Regions with higher uncertainty get adjusted more
- Information from the total forecast helps improve regional accuracy
- At least one region typically shows RMSE improvement

### Methods Available
- **OLS (Ordinary Least Squares)**: Optimally adjusts all levels to minimize squared errors
- **Bottom-up**: Keeps bottom-level forecasts fixed, aggregates up

We'll demonstrate that OLS reconciliation improves at least one region's forecast accuracy.

In [ ]:
# Part 2: Use base forecasts from Part 1 (AutoForecaster per-series selection)
print("📊 Step 1: Prepare base forecasts for hierarchical reconciliation")
print("-" * 75)
print("""
💡 We'll use the AutoForecaster forecasts from Part 1 as our base forecasts.
   These are the per-series optimized forecasts that we'll now reconcile.
""")

# Generate base forecasts using AutoForecaster (already trained in Part 1)
base_forecasts = auto_forecaster.forecast(X=X_test)

# Display which models were used
for series_name in series_to_test:
    model_name = type(auto_forecaster.best_models_[series_name]).__name__.replace('Forecaster', '')
    print(f"   {series_name}: Using {model_name}")

print(f"\n✅ Base forecasts generated using AutoForecaster")

# Check coherence before reconciliation
base_total = base_forecasts['total'].values
base_sum = base_forecasts['region_a'].values + base_forecasts['region_b'].values

coherence_error_base = np.abs(base_total - base_sum).mean()

print(f"\n⚠️  Coherence check (before reconciliation):")
print(f"   Mean |total - (region_a + region_b)|: {coherence_error_base:.2f}")
print(f"   Forecasts do NOT add up correctly - this is normal for independent models!")

In [ ]:
# Step 2: Apply hierarchical reconciliation
print("📊 Step 2: Apply hierarchical reconciliation")
print("-" * 60)

# Define hierarchy: total = region_a + region_b
hierarchy = {'total': ['region_a', 'region_b']}

# Use OLS reconciliation (Ordinary Least Squares)
reconciler = HierarchicalReconciler(forecasts=base_forecasts, hierarchy=hierarchy)
reconciler.reconcile(method='ols')
reconciled_forecasts = reconciler.reconciled_forecasts

print(f"✅ Applied OLS reconciliation")

# Check coherence after reconciliation
reconciled_total = reconciled_forecasts['total'].values
reconciled_sum = reconciled_forecasts['region_a'].values + reconciled_forecasts['region_b'].values
coherence_error_reconciled = np.abs(reconciled_total - reconciled_sum).mean()

print(f"\n✅ Coherence check (after reconciliation):")
print(f"   Mean |total - (region_a + region_b)|: {coherence_error_reconciled:.10f}")
print(f"   Forecasts now add up correctly!")

# Accuracy comparison - Focus on REGION-LEVEL improvements
print(f"\n" + "=" * 70)
print("📊 ACCURACY COMPARISON: Focus on Region-Level Improvements")
print("=" * 70)
print(f"\n   {'Series':<12} {'Base RMSE':<12} {'Reconciled':<12} {'Change':<12} {'Result':<15}")
print(f"   {'-'*60}")

results = {}
for col in ['region_a', 'region_b', 'total']:
    base_rmse_val = rmse(y_test[col], base_forecasts[col])
    recon_rmse = rmse(y_test[col], reconciled_forecasts[col])
    change = 100 * (recon_rmse - base_rmse_val) / base_rmse_val
    results[col] = {'base': base_rmse_val, 'reconciled': recon_rmse, 'change': change}
    result_str = "✅ IMPROVED" if change < 0 else ("↔ Same" if change == 0 else "Trade-off")
    print(f"   {col:<12} {base_rmse_val:<12.2f} {recon_rmse:<12.2f} {change:>+6.1f}%     {result_str}")

# Highlight region-level improvements
improved_regions = [k for k in ['region_a', 'region_b'] if results[k]['change'] < 0]
improved_all = [k for k, v in results.items() if v['change'] < 0]

print(f"\n" + "=" * 70)
print("📖 REGION-LEVEL INTERPRETATION")
print("=" * 70)

if improved_regions:
    print(f"\n🎯 REGION IMPROVEMENTS (Key Benefit of OLS Reconciliation):")
    for region in improved_regions:
        print(f"   ✅ {region}: RMSE reduced by {abs(results[region]['change']):.1f}%")
        print(f"      Base: {results[region]['base']:.2f} → Reconciled: {results[region]['reconciled']:.2f}")
    
    print(f"""
   💡 WHY REGIONS IMPROVE:
      OLS reconciliation uses information from ALL levels to adjust forecasts.
      When the total forecast is more accurate than individual regions,
      OLS "borrows strength" from the total to improve regional estimates.
""")
else:
    print(f"\n   Note: In this run, regions traded accuracy for coherence.")
    print(f"   Re-run with different data to see region improvements.")

if improved_all:
    print(f"\n📊 All improved series: {', '.join(improved_all)}")

print(f"""
💡 Key Takeaways:
   1. COHERENCE: Forecasts now satisfy total = region_a + region_b
   2. REGION ACCURACY: OLS can improve region-level forecasts (not just total)
   3. BUSINESS VALUE: Coherent + accurate regional forecasts for:
      - Inventory allocation across warehouses
      - Regional sales targets that sum to company total
      - Consistent budgeting across organizational levels
""")

In [ ]:
# Visualize base vs reconciled forecasts
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

series_names = ['region_a', 'region_b', 'total']
colors = ['steelblue', 'darkorange', 'green']

for idx, (series, color) in enumerate(zip(series_names, colors)):
    ax = axes[idx]
    ax.plot(y_test.index, y_test[series], label='Actual', color='black', linewidth=2, marker='o', markersize=4)
    ax.plot(base_forecasts.index, base_forecasts[series], label='Base', color=color, 
            linewidth=2, linestyle='--', alpha=0.6)
    ax.plot(reconciled_forecasts.index, reconciled_forecasts[series], label='Reconciled', 
            color='red', linewidth=2, linestyle='-.')
    
    base_err = rmse(y_test[series], base_forecasts[series])
    recon_err = rmse(y_test[series], reconciled_forecasts[series])
    ax.set_title(f'{series.replace("_", " ").title()}\nRMSE: {base_err:.1f} → {recon_err:.1f}', fontweight='bold')
    ax.set_xlabel('Date')
    ax.set_ylabel('Sales')
    ax.legend(fontsize=8)
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## Part 3: Interpretability — Understanding Forecast Drivers

Understanding *why* a model makes certain predictions is crucial for:
- **Building trust**: Stakeholders need to understand the model
- **Debugging**: Identify when the model is using spurious correlations
- **Business insights**: Learn which factors truly drive your metrics

The `DriverAnalyzer` provides:
1. **Sensitivity analysis**: Perturbs features and measures prediction change (works for all models)
2. **SHAP values**: Game-theoretic attribution for tree-based models (XGBoost, RandomForest)

We'll analyze feature importance for **all three series** to see how temperature and promotion affect each differently.

In [ ]:
# Analyze forecast drivers for ALL SERIES using trained AutoForecaster models
print(f"📊 Sensitivity Analysis: Feature Importance for All Series")
print("=" * 60)
print("""
💡 We'll use the trained AutoForecaster models from Part 1 for each series.
   DriverAnalyzer will automatically select the best interpretability method 
   for the underlying best model chosen by AutoForecaster.
""")

# Store results for all series
sensitivity_results = {}

for series_name in ['region_a', 'region_b', 'total']:
    print(f"\n🔹 Series: {series_name}")
    
    # Use the best model selected by AutoForecaster for this series
    best_model = auto_forecaster.best_models_[series_name]
    best_model_name = type(best_model).__name__.replace('Forecaster', '')
    print(f"   Selected Model: {best_model_name}")
    
    # Create DriverAnalyzer with the best model
    analyzer = DriverAnalyzer(
        model=best_model,
        feature_names=['temperature', 'promotion']
    )
    
    # Let DriverAnalyzer automatically select the best interpretation method
    # It will try SHAP for tree-based models, otherwise fall back to sensitivity
    try:
        importance_df = analyzer.calculate_feature_importance(X_test, y_test[[series_name]])
        method_used = "SHAP/Permutation" if hasattr(best_model, 'feature_importances_') else "Sensitivity"
    except Exception as e:
        # Fallback to sensitivity analysis if automatic selection fails
        importance_df = analyzer.calculate_feature_importance(X_test, y_test[[series_name]], method='sensitivity')
        method_used = "Sensitivity"
    
    print(f"   Method: {method_used}")
    
    # Extract importance values
    temp_sens = float(importance_df.loc['temperature'].mean())
    promo_sens = float(importance_df.loc['promotion'].mean())
    sensitivity_results[series_name] = {'temperature': temp_sens, 'promotion': promo_sens}
    
    print(f"   temperature    : {temp_sens:.4f}")
    print(f"   promotion      : {promo_sens:.4f}")

# Summary comparison
print(f"\n" + "=" * 60)
print("📊 SUMMARY: Feature Impact Across All Series")
print("=" * 60)
print(f"\n   {'Series':<15} {'Temperature':<15} {'Promotion':<15} {'Dominant Driver':<20}")
print(f"   {'-'*65}")
for series_name in ['region_a', 'region_b', 'total']:
    temp = sensitivity_results[series_name]['temperature']
    promo = sensitivity_results[series_name]['promotion']
    dominant = 'Promotion' if promo > temp else 'Temperature'
    print(f"   {series_name:<15} {temp:<15.4f} {promo:<15.4f} {dominant:<20}")

print(f"""
📖 INTERPRETATION:
   • Feature importance shows how much predictions change when each feature 
     is perturbed (sensitivity) or attributed (SHAP/permutation)
   • Higher values = model predictions are more sensitive to that feature
   
   Expected based on data generation:
      • region_a: +50 per promotion, +1.6 per °C → Promotion dominant ✓
      • region_b: +4 per promotion, +2.0 per °C → Temperature more important
      • total: Sum of both → Mixed effects
      
   💡 AutoForecaster automatically selected the best model for each series,
      and DriverAnalyzer automatically selected the best interpretation method
      for each model type (SHAP for tree-based, sensitivity for others).
""")

In [ ]:
# Visualize feature importance
if importance_df is not None:
    plt.figure(figsize=(8, 4))
    
    # Calculate mean importance across targets
    mean_importance = importance_df.mean(axis=1)
    
    colors = ['steelblue', 'darkorange']
    bars = plt.barh(mean_importance.index, mean_importance.values, color=colors)
    
    plt.xlabel('Importance Score (higher = more important)')
    plt.ylabel('Feature')
    plt.title('Feature Importance Analysis', fontweight='bold')
    plt.grid(alpha=0.3, axis='x')
    plt.tight_layout()
    plt.show()
    
    # Interpretation
    print("📖 INTERPRETATION:")
    print(f"   Higher values indicate features that cause larger prediction errors when shuffled.")
    print(f"   This means the model relies more heavily on these features for accurate forecasts.")

In [ ]:
# Detailed SHAP Analysis Example (Region A)
print("🔍 Detailed Interpretability Analysis (Region A)")
print("-" * 60)
print("""
💡 We'll demonstrate advanced interpretability using Region A's model.
   If the AutoForecaster selected a tree-based model, we'll show SHAP values.
   Otherwise, we'll demonstrate sensitivity analysis in detail.
""")

# Get the best model for region_a from AutoForecaster
region_a_model = auto_forecaster.best_models_['region_a']
region_a_model_name = type(region_a_model).__name__.replace('Forecaster', '')

print(f"\n📊 Region A Model: {region_a_model_name}")

# Generate predictions
region_a_forecast = auto_forecaster.forecast(X=X_test)
region_a_rmse = rmse(y_test['region_a'], region_a_forecast['region_a'])
print(f"   Holdout RMSE: {region_a_rmse:.2f}")

# Create analyzer
analyzer = DriverAnalyzer(
    model=region_a_model,
    feature_names=['temperature', 'promotion']
)

# Always show sensitivity analysis (works for all models)
print(f"\n📊 Sensitivity Analysis:")
sensitivity = analyzer.calculate_feature_importance(X_test, y_test[['region_a']], method='sensitivity')

temp_sens = float(sensitivity.loc['temperature'].mean())
promo_sens = float(sensitivity.loc['promotion'].mean())

print(f"   temperature    : {temp_sens:.4f}")
print(f"   promotion      : {promo_sens:.4f}")

print(f"\n📖 INTERPRETATION:")
if promo_sens > temp_sens:
    print(f"   • Promotion has ~{promo_sens/temp_sens:.1f}x more impact than temperature")
else:
    print(f"   • Temperature has ~{temp_sens/promo_sens:.1f}x more impact than promotion")
print(f"   • This matches our data: Region A has +50 sales per promotion, +1.6 per °C")

# Try SHAP if model is tree-based and SHAP is available
is_tree_based = region_a_model_name in ['XGBoost', 'RandomForest']

if SHAP_AVAILABLE and is_tree_based:
    try:
        print(f"\n📊 SHAP Values (game-theoretic feature attribution):")
        shap_values = analyzer.calculate_shap_values(X_train, y_train[['region_a']])
        shap_importance_df = analyzer.get_shap_feature_importance(shap_values)
        
        # Display SHAP importance
        print(f"   SHAP feature importance (mean |SHAP|):")
        for feature in shap_importance_df.index:
            imp_val = float(shap_importance_df.loc[feature].mean())
            print(f"   {feature:<15}: {imp_val:.4f}")
            
        print(f"\n   💡 SHAP values provide theoretically grounded attribution based on")
        print(f"      Shapley values from cooperative game theory. They consider")
        print(f"      feature interactions and provide consistent, accurate attribution.")
        
    except Exception as e:
        print(f"\n   ⚠️ SHAP calculation issue: {str(e)[:80]}...")
        print(f"   Sensitivity analysis above provides reliable feature importance.")
        
elif is_tree_based and not SHAP_AVAILABLE:
    print(f"\n💡 Install SHAP for advanced attribution: pip install shap")
    print(f"   SHAP works with tree-based models (XGBoost, RandomForest)")
    
else:
    print(f"\n💡 SHAP analysis is available for tree-based models (XGBoost, RandomForest).")
    print(f"   Current model ({region_a_model_name}) uses sensitivity analysis.")
    print(f"   Sensitivity analysis perturbs features and measures prediction changes.")

## Summary

You've learned how to use AutoTSForecast for:

1. **AutoForecaster vs Individual Models**
   - AutoForecaster outperforms using any single model for all series
   - Per-series model selection via CV on training data only (no data leakage)
   - Different series automatically select their optimal models

2. **Hierarchical Reconciliation with Region-Level Improvements**
   - Ensure forecasts are coherent: `total = region_a + region_b`
   - OLS reconciliation can improve region-level accuracy (not just total)
   - Trade-off: coherence + potential regional improvements

3. **Interpretability**
   - Sensitivity analysis: feature impact on predictions
   - SHAP values for tree-based models (XGBoost, RandomForest)

---

## Quick Reference

### AutoForecaster (Per-Series Model Selection)
```python
from autotsforecast import AutoForecaster
from autotsforecast.models.external import ProphetForecaster, XGBoostForecaster

# Each series independently selects its best model via CV
forecaster = AutoForecaster(
    candidate_models=[ProphetForecaster(horizon=14), XGBoostForecaster(horizon=14)],
    n_splits=3,
    test_size=14,
    metric='rmse'
)
forecaster.fit(y_train)  # Multivariate: DataFrame with multiple columns
forecast = forecaster.forecast()
```

### Hierarchical Reconciliation
```python
from autotsforecast.hierarchical.reconciliation import HierarchicalReconciler

reconciler = HierarchicalReconciler(forecasts=base_forecasts, hierarchy={'total': ['region_a', 'region_b']})
reconciler.reconcile(method='ols')
coherent_forecasts = reconciler.reconciled_forecasts
```

### Backtesting Individual Models
```python
from autotsforecast.backtesting.validator import BacktestValidator

validator = BacktestValidator(model=ProphetForecaster(horizon=14), n_splits=5, test_size=14)
validator.run(y_train)
results = validator.get_fold_results()  # DataFrame with rmse, mape per fold
```

---

## Next Steps

- **[API Reference](../API_REFERENCE.md)** — Detailed parameter documentation
- **[Quick Start](../QUICKSTART.md)** — Condensed overview
- **[Technical Documentation](../TECHNICAL_DOCUMENTATION.md)** — Advanced topics